In [12]:
!pip install ckiptagger
!pip install fuzzy
!pip install fuzzychinese
from ckiptagger import data_utils, WS, NER, POS
data_utils.download_data_gdown("./")

67.1MB [00:15, 91.5MB/s]Downloading...
From: https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
To: /content/data.zip

0.00B [00:00, ?B/s]
16.3MB [00:00, 161MB/s]
41.9MB [00:00, 216MB/s]
69.7MB [00:00, 243MB/s]
96.5MB [00:00, 253MB/s]
122MB [00:00, 241MB/s] 
147MB [00:00, 187MB/s]
167MB [00:00, 191MB/s]
188MB [00:00, 188MB/s]
208MB [00:01, 188MB/s]
227MB [00:01, 170MB/s]
245MB [00:01, 151MB/s]
261MB [00:01, 139MB/s]
276MB [00:01, 132MB/s]
290MB [00:01, 135MB/s]
309MB [00:01, 148MB/s]
329MB [00:01, 162MB/s]
355MB [00:02, 188MB/s]
380MB [00:02, 205MB/s]
405MB [00:02, 216MB/s]
428MB [00:02, 222MB/s]
451MB [00:02, 176MB/s]
470MB [00:02, 178MB/s]
491MB [00:02, 185MB/s]
514MB [00:02, 198MB/s]
540MB [00:02, 214MB/s]
563MB [00:03, 162MB/s]
581MB [00:03, 168MB/s]
600MB [00:03, 171MB/s]
619MB [00:03, 173MB/s]
640MB [00:03, 182MB/s]
661MB [00:03, 188MB/s]
682MB [00:03, 194MB/s]
708MB [00:03, 214MB/s]
735MB [00:03, 227MB/s]
758MB [00:04, 216MB/s]
780MB [00:04, 189MB/s]
800MB [00:04, 

# 1.閒聊問答實作

請匯入fuzzy套件，並嘗試判斷與下列語句的相似度:
```
['最近有什麼好看的電影','推薦不錯的電影','那些影片評價不錯', "高評價的電影", "很棒的影片"]
```

若相似分數 >= 70，則回覆:

```
"最近當然要看鬼滅劇場版阿!"
```

In [13]:
# rule based chatbot using fuzzy match
from fuzzychinese import FuzzyChineseMatch
query = input(str("問我一個問題吧！\n")) # input query (asking for movie recommendations)
sentence = ['最近有什麼好看的電影','推薦不錯的電影','那些影片評價不錯', "高評價的電影", "很棒的影片"] # sentences to calculate fuzzy similarity

def get_similarity(query, sentence):
  fcm = FuzzyChineseMatch(ngram_range=(3, 3), analyzer='stroke')
  fcm.fit(sentence) # fit the possible sentences into the model
  fcm.transform([query])
  max_similarity = fcm.get_similarity_score().max()

  return max_similarity.round(2)

def answer(max_similarity, thresh = 0.7):
  # if the similarity is > .70, then return a recommendation for movie
  if max_similarity >= thresh:
    return "最近當然要看鬼滅劇場版阿!"
  else:
    return "我不太懂你在說啥耶"

answer(get_similarity(query, sentence), thresh = 0.7)

問我一個問題吧！
最近有什麼不錯的電影


'最近當然要看鬼滅劇場版阿!'

In [14]:
import requests
question = str(input("問我一個問題吧！"))
def get_api_ans(question):
    URL = "http://api.qingyunke.com/api.php?key=free&appid=0&msg={}".format(question)
    headers = {'Content-Type': 'application/json'}
    res = requests.get(URL, headers=headers)
    return res.json()['content']

get_api_ans(question)

問我一個問題吧！我想吃冰


'那就快去吃啊'

In [22]:
greating = "你是誰"
to_girl = [("請管好你的嘴","為什麼","因為我會隨時親你"),
           ("你單身多久了","一年多了","對不起，讓你等了這麼久"),
           ("我真的是太笨了","怎麼啦","除了喜歡你，其它什麼都做不好")]
def get_cutomize_ans(query):
    # 完全比對
    if query == greating:
        return "我是你的助理! 我可以陪你聊天喔~"

    # 部分比對
    elif "撩妹" in query:
        return str(random.choice(to_girl))

    # 模糊比對-1
    elif "台北" in query and "景點" in query:
        return "有101, 中正紀念堂還可以去爬象山喔!"
    
    elif any(i in query for i in "台北 台北市 台灣首都 天龍國".split()):
        if any(j in query for j in "手搖飲料店 珍奶店 飲料店".split()):
            url = "https://google.com/search?hl=zh-TW&q=台北市手搖飲料店&num=10"
            return url
        elif any(j in query for j in "名產 伴手禮 土產".split()):
            url = "https://google.com/search?hl=zh-TW&q=台北市名產&num=10"
            return url
        
        else:
            return None
            # return "我還沒辦法回覆你的問題喔!"
    else:
        return None
        # return "我還沒辦法回覆你的問題喔!"

# 2.知識問答實作

請嘗試使用 regular expression 取得 html 的右側資訊摘要 (president.png)
並將其作為人物百科的回覆，已解決目前 parsing 結果的不足 (error.png)

In [15]:
import jieba
import jieba.posseg
import urllib.parse
import requests, re
from bs4 import BeautifulSoup

ws = WS('./data')
pos = POS('./data')
ner = NER('./data')

/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:901: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/usr/local/lib/python3.7/dist-packages/keras/engine/base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [16]:
def named_entity_checker(query):
  '''
  recognize named entity in query
  '''
  segmentation = ws([query])
  pos_tagging = pos(segmentation)
  named_entity = ner(segmentation, pos_tagging)
  return any([entity[2] == 'PERSON' for entity in named_entity[0]])


# 進行斷詞
def jieba_tokenizer(query):
    word_pos = jieba.posseg.cut(query)
    return [(word,pos) for word, pos in word_pos]

# 取得 html   
def get_html(query, num):
    query = urllib.parse.quote_plus(query)
    url = 'https://google.com/search?hl=zh-TW&q=' + query + "&num=" + str(num)
    try:
        response = requests.get(url)
        return BeautifulSoup(response.text, 'lxml')
    except:
        return None
    
# 取得搜尋結果的標題
def get_title(text):
    title = re.findall(r'<h3 class="zBAuLc"><div class="BNeawe vvjwJb AP7Wnd">(?P<content>.*?)</div></h3>', str(text), re.DOTALL)
    if title != []:
        title = title[0].replace('\n', '')
    return title

# 取得結果摘要
def get_content(text):
    content = re.findall(r'<div class="BNeawe s3v9rd AP7Wnd"><div><div class="BNeawe s3v9rd AP7Wnd">(?P<content>.*?)<span>', str(text), re.DOTALL)
    if content != []:
        content = content[0].replace('\n', '')
        content = re.sub(r'<span class="BNeawe">.*?<div class="BNeawe s3v9rd AP7Wnd">', "", content)
    return content

def get_encyclopedia_ans(query):
    token = jieba_tokenizer(query)
    html = get_html(query, 1)
    candidate = html.select('div.ZINbbc.xpd.O9g5cc.uUPGi')
    answer = get_content(html) if named_entity_checker(query) else None
    return answer

In [19]:
person = str(input('你想知道哪個人物的資訊呢？\n'))
get_encyclopedia_ans(person)

你想知道哪個人物的資訊呢？
馬英九


'馬英九，中華民國政治人物、學者，曾任第12、13任中華民國總統、第4、6任中國國民黨主席等職。祖籍江西省永新縣，籍貫湖南省衡山縣，馬鶴凌和秦厚修之子，生於英屬香港九龍油麻地，1952年隨雙親定居臺灣臺北市。畢業於國立臺灣大學法律學系，其後前往美國深造，獲紐約大學法學碩士學位、哈佛大學司法學博士學位。 '

# 3.聊天機器人answer selection

請嘗試加入"形容詞"及"副詞"的判斷，以優化"閒聊"與"知識"問答的分類效果

In [30]:
#optimize answer selection based on part of speech (adding adjs and advs)
queries = [
    "蔡英文是誰", 
    "介紹陳零九", 
    "柴可夫斯基", 
    "蘋果為什麼會變紅", 
    "為什麼這群人是網紅霸主", 
    "熱氣球的原理",   
    
#     "馬英九是誰", 
#     "介紹吳宗憲", 
#     "拜登的生平", 
#     "含羞草的原理", 
#     "黑死病發生的時間", 
#     "海豚的智商",  
    
    "你不是已經在家了嗎",
    "重點是什麼呢", # @@
    "我最近睡不著",
    "你聽過鬼滅之刃嗎",  # @@
    "一起去玩啦",
    "風大雨大太陽大",
    "我好想吃冰喔",  # @@
    "呵呵，如果是這樣就沒救囉",
    "佛祖會生氣喔",  # @@
    "你會溜直排輪嗎",
    
#     "你是誰",
#     "撩妹金句",
#     "告訴我台北的知名景點有哪些",
#     "景點有哪些",
#     "台北市有幾間手搖飲料店",
#     "台北市有幾間搖手店",
#     "天龍國的伴手禮去哪裡買",
#     "天龍人的伴手禮"
]

def ckip_tokenizer(query):
    word_list = ws([query])
    pos_list = pos(word_list)

    return list(zip(word_list[0], pos_list[0]))

def query_classifier(token):
    '''
    return
     True => samll talk
     False => encyclopedia
    '''
    count_nouns, count_verbs, count_advs, count_adjs = 0, 0, 0, 0

    token_len = len(token)
    for i, token in enumerate(token):
        word, pos = token

        if pos == "x": # 標點符號
            token_len -= 1
                      
        if any(pos == p for p in "Na Nb Nc Nh".split(" ")): # 名詞類
            count_nouns += 1
        elif pos[0] == "V": # 動詞類
            count_verbs += 1
        elif pos[0] == "D":
            count_advs += 1
        elif pos[0] == "A":
          count_adjs += 1


    
    # 若完全沒比對出名詞 可能為口語短句 應忽略
    if count_nouns == 0:
        return True
     # 若動詞超過兩個 可能為非口語短句 不應忽略
    elif count_verbs >= 2:
        return False
    else:
        return False
    
def get_response(query):
    token = ckip_tokenizer(query)
    res = query_classifier(token)
    
    cutomize_ans = get_cutomize_ans(query)
    # 先判斷是否為自定義語句
    if cutomize_ans != None:
        return cutomize_ans
    else:
      # to avoid returning all none answers
      response = get_encyclopedia_ans(query)
        # 閒聊問句
      if res or response == None:
          return get_api_ans(query)
        # 知識問句
      else:
          return response
import time
for query in queries:
    response = get_response(query)
    print("Q:",query)
    print("A:",response)
    print("===")
    #time.sleep(1)

Q: 蔡英文是誰
A: 蔡英文，中華民國民主進步黨籍政治人物，現任中華民國總統兼民主進步黨主席，亦為法律學者與律師，曾擔任過行政院副院長、立法委員、大陸委員會主任委員、國家安全會議諮詢委員等職務。生於臺北市中山區，本籍屏東縣枋山鄉，是客家人和臺灣原住民族排灣族之後裔。 
===
Q: 介紹陳零九
A: 陳零九，本名為陳志豪，出生於台灣高雄，台灣創作歌手、主持人。2009年起成為網路歌手，2013年起簽約滾石唱片，截至2021年8月31日已發行7張正規專輯和1張數位專輯。現為《娛樂百分百》主持人、雙人組合「九澤CP」及男子團體「五堅情」的成員。 
===
Q: 柴可夫斯基
A: 彼得·伊利奇·柴可夫斯基，俄國浪漫樂派作曲家，其作品有民族樂派特徵，但仍以浪漫風格為基準。其風格直接和間接地影響了很多後來者。柴可夫斯基出生於沃特金斯克一個貴族家庭，從小在母親的教導下學習鋼琴，由於父親的反對，進入法學院學習，畢業以後在法院工作。 
===
Q: 蘋果為什麼會變紅
A: 麻麻呀...又来一个说瞎话的
===
Q: 為什麼這群人是網紅霸主
A: 我就一个脑袋，思想再活跃也有局限性
===
Q: 熱氣球的原理
A: 呢D機械野你唔識?啦
===
Q: 你不是已經在家了嗎
A: 帥哥，又有錢
===
Q: 重點是什麼呢
A: 什麼？
===
Q: 我最近睡不著
A: 我不睡，精神着呢
===
Q: 你聽過鬼滅之刃嗎
A: 知道切糕不，一刀上海买房 两刀杨幂上床 三刀盖茨认娘 四刀铁定入常
===
Q: 一起去玩啦
A: 好啊，一块去！
===
Q: 風大雨大太陽大
A: 别吵，伦家正在梅州行网站上逛论坛呢
===
Q: 我好想吃冰喔
A: Cold，stone超棒
===
Q: 呵呵，如果是這樣就沒救囉
A: 嘿嘿
===
Q: 佛祖會生氣喔
A: {face:3}有些同事反映你，白天文明，但不精神，晚上精神，但不文明
===
Q: 你會溜直排輪嗎
A: 宝贝，你说的话好深奥啊，有点像哥德巴赫猜想，暂时无解！ 
===
